In [1]:
#!/usr/bin/env julia

# Python에서 쓰이던 method들을 Julia에서 이름만 적절히 바꾸면 잘 되는 경우가 많다. 

In [48]:
using Yao
using Yao.ConstGate # needed for P1 = 0.5*(I - sigma_z) block , P1 is n_i matrix

## H(t) = Ω(t) ∑_i σ_i^x - δ(t) ∑_i n_i + u ∑_ij n_i n_j

In [3]:
const u = 1.35 # const is a global variable whose values will not change. In introduction paper, u is choosed by 1.35.
const Ω_max = 1.89
const δ_0 = -1.0
const δ_max = 1.0

1.0

In [4]:
# This function makes graph that satisfy constrain for UD-MIS problem
function get_edges(graph::Vector{NTuple{2, Float64}}) # input data is coordinate for each vertex
    Nv = size(graph)[1] # size(graph)[1] is same as size(graph,1) = number of vertex
    edges = falses(Nv, Nv) #falses make (nv*nv) matrix that has all 0 elements. In this matrix, graph was created. 
    for i in 1:(Nv-1)
        xi, yi = graph[i]
        for j in (i+1):Nv
            xj, yj = graph[j]

            dij = sqrt((xi - xj)^2. + (yi - yj)^2.)
            if dij <= 1.0
                edges[i,j] = true
            end
        end
    end
    return findall(edges)  ## findall give back location of vertex that have edge. findall's return value is Vector{CartesianIndex} because edges are n*n matrix. 
end

get_edges (generic function with 1 method)

In [5]:
function Ω(t::Float64)
    if 0 <= t <= 0.25
        return 4. * Ω_max * t
    elseif 0.25 < t <= 0.69
        return Ω_max
    elseif 0.69 < t <= 1
        return Ω_max - Ω_max * (t-0.69) / 0.31 ## ??? discontinuous???
    end
end

Ω (generic function with 1 method)

In [6]:
function δ(t::Float64)
    if 0 <= t <= 0.25
        return δ_0
    elseif 0.25 < t <= 0.69
        return (t-0.25) * (δ_max - δ_0)/0.44 + δ_0 
    elseif 0.69 < t <= 1
        return δ_max
    end
end 

δ (generic function with 1 method)

In [7]:
function hamiltonian(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, t::Float64)
    # the UD-MIS Hamiltonian
    Nv = size(graph)[1] # number of vertices

    interaction_term = map(1:size(edges)[1]) do i
        l,m = edges[i][1], edges[i][2] # saving nodes that have edges.
        repeat(Nv,u*P1,(l,m)) # Yao's reapeat ftn. (Nv:number of qubit,u*P1 : gate,(l,m) : location repeated)
    end |> sum #piping, pipe
    interaction_term - δ(t)*sum(map(i->put(Nv,i=>P1), 1:Nv)) + Ω(t)*sum(map(i->put(Nv,i=>X), 1:Nv)) #put 원하는 장소에 원하는 gate 넣은 함수
end

hamiltonian (generic function with 1 method)

In [8]:
function run_annealing(graph::Vector{NTuple{2, Float64}}, edges::Vector{CartesianIndex{2}}, dt::Float64)
    psi_t = zero_state(size(graph)[1])  # 왜 0에서 propagate 시키더라?
    for t in 0:dt:1.0
        h = hamiltonian(graph, edges, t)
        psi_t = psi_t |> TimeEvolution(h, dt)
    end
    return psi_t
end

run_annealing (generic function with 1 method)

In [9]:
graph = [(0.3461717838632017, 1.4984640297338632), (0.6316400411846113, 2.5754677320579895), (1.3906262250927481, 2.164978861396621), (0.66436005100802, 0.6717919819739032), (0.8663329771713457, 3.3876341010035995), (1.1643107343501296, 1.0823066243402013)]
edges = get_edges(graph)
dt = 0.001

0.001

In [39]:
psi = run_annealing(graph, edges, dt)
n_shots = 4096
samples = measure(psi; nshots=n_shots)

4096-element Vector{BitBasis.BitStr64{6}}:
 101111 ₍₂₎
 110111 ₍₂₎
 111101 ₍₂₎
 110111 ₍₂₎
 111110 ₍₂₎
 110101 ₍₂₎
 111101 ₍₂₎
 011111 ₍₂₎
 001111 ₍₂₎
 111111 ₍₂₎
 111111 ₍₂₎
 111110 ₍₂₎
 111101 ₍₂₎
          ⋮
 110101 ₍₂₎
 001111 ₍₂₎
 111111 ₍₂₎
 110101 ₍₂₎
 011111 ₍₂₎
 110111 ₍₂₎
 011111 ₍₂₎
 011111 ₍₂₎
 011111 ₍₂₎
 111110 ₍₂₎
 111111 ₍₂₎
 111110 ₍₂₎

In [40]:
@show samples

11 ₍₂₎, 111011 ₍₂₎, 110111 ₍₂₎, 111011 ₍₂₎, 110011 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111101 ₍₂₎, 101111 ₍₂₎, 101010 ₍₂₎, 111111 ₍₂₎, 110111 ₍₂₎, 111111 ₍₂₎, 111100 ₍₂₎, 111111 ₍₂₎, 001111 ₍₂₎, 011111 ₍₂₎, 111111 ₍₂₎, 100111 ₍₂₎, 111110 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 110111 ₍₂₎, 111100 ₍₂₎, 111111 ₍₂₎, 111110 ₍₂₎, 011111 ₍₂₎, 111111 ₍₂₎, 111101 ₍₂₎, 111111 ₍₂₎, 111110 ₍₂₎, 011111 ₍₂₎, 111101 ₍₂₎, 010110 ₍₂₎, 001111 ₍₂₎, 011111 ₍₂₎, 111111 ₍₂₎, 111110 ₍₂₎, 111111 ₍₂₎, 111101 ₍₂₎, 001001 ₍₂₎, 111110 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 011101 ₍₂₎, 110101 ₍₂₎, 110101 ₍₂₎, 110111 ₍₂₎, 111111 ₍₂₎, 011111 ₍₂₎, 111111 ₍₂₎, 111101 ₍₂₎, 111110 ₍₂₎, 110111 ₍₂₎, 101111 ₍₂₎, 111111 ₍₂₎, 101110 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 110111 ₍₂₎, 011111 ₍₂₎, 111100 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 110111 ₍₂₎, 011101 ₍₂₎, 011111 ₍₂₎, 110111 ₍₂₎, 101111 ₍₂₎, 110111 ₍₂₎, 111110 ₍₂₎, 111110 ₍₂₎, 111111 ₍₂₎, 111111 ₍₂₎, 110111 ₍₂₎, 111111 ₍₂₎, 011111 ₍

4096-element Vector{BitBasis.BitStr64{6}}:
 101111 ₍₂₎
 110111 ₍₂₎
 111101 ₍₂₎
 110111 ₍₂₎
 111110 ₍₂₎
 110101 ₍₂₎
 111101 ₍₂₎
 011111 ₍₂₎
 001111 ₍₂₎
 111111 ₍₂₎
 111111 ₍₂₎
 111110 ₍₂₎
 111101 ₍₂₎
          ⋮
 110101 ₍₂₎
 001111 ₍₂₎
 111111 ₍₂₎
 110101 ₍₂₎
 011111 ₍₂₎
 110111 ₍₂₎
 011111 ₍₂₎
 011111 ₍₂₎
 011111 ₍₂₎
 111110 ₍₂₎
 111111 ₍₂₎
 111110 ₍₂₎

In [41]:
samples[1]

101111 ₍₂₎

In [50]:
Save = zeros(Int64, n_shots) # task1을 할때처럼 여기에 이진수 숫자를 10진수로 바꿔서 저장하고, 갯수를 세서 prob를 만든 다음에 plotting을 할 생각이다. 

4096-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [52]:
Num = zeros(Int64, 2^size(graph)[1]) #세어진 각각의 갯수를 저장할 vector

64-element Vector{Int64}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0

In [68]:
Pr = zeros(2^size(graph)[1])

64-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [43]:
typeof(samples[1])

BitBasis.BitStr64{6} (alias for BitBasis.BitStr{6, Int64})

In [53]:
Int64(samples[1]) # samples라는 이진수 타입을 10진수로 표현함. 

47

## Binary to Decimal

In [57]:
for i in 1:n_shots
    Save[i] = Int64(samples[i])
end

In [58]:
Save

4096-element Vector{Int64}:
 47
 55
 61
 55
 62
 53
 61
 31
 15
 63
 63
 62
 61
  ⋮
 53
 15
 63
 53
 31
 55
 31
 31
 31
 62
 63
 62

In [61]:
for i in 1:n_shots
    Num[Save[i] + 1] += 1
end

In [63]:
@show Num

Num = [0, 0, 0, 0, 0, 0, 2, 1, 0, 3, 0, 8, 0, 4, 1, 51, 0, 0, 4, 1, 6, 4, 29, 9, 0, 1, 1, 39, 1, 140, 6, 454, 0, 1, 0, 3, 0, 0, 0, 51, 1, 4, 6, 13, 5, 5, 44, 172, 0, 2, 2, 56, 1, 136, 6, 425, 3, 3, 49, 155, 126, 361, 431, 1270]


64-element Vector{Int64}:
    0
    0
    0
    0
    0
    0
    2
    1
    0
    3
    0
    8
    0
    ⋮
    1
  136
    6
  425
    3
    3
   49
  155
  126
  361
  431
 1270

In [64]:
idx = 0
for i in 1:2^size(graph)[1]
    idx += Num[i]
end

In [65]:
idx

4096

In [69]:
for i in 1:2^size(graph)[1]
    Pr[i]=Num[i]/n_shots
end

Pr

64-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.00048828125
 0.000244140625
 0.0
 0.000732421875
 0.0
 0.001953125
 0.0
 ⋮
 0.000244140625
 0.033203125
 0.00146484375
 0.103759765625
 0.000732421875
 0.000732421875
 0.011962890625
 0.037841796875
 0.03076171875
 0.088134765625
 0.105224609375
 0.31005859375